# Домашнее задание № 4. Языковые модели

## Задание 1 (8 баллов).

В семинаре для генерации мы использовали предположение маркова и считали, что слово зависит только от 1 предыдущего слова. Но ничто нам не мешает попробовать увеличить размер окна и учитывать два или даже три прошлых слова. Для них мы еще сможем собрать достаточно статистик и, логично предположить, что качество сгенерированного текста должно вырасти.

Попробуйте сделать языковую модель, которая будет учитывать два предыдущих слова при генерации текста.
Сгенерируйте несколько текстов (3-5) и расчитайте перплексию получившейся модели. 
Можно использовать данные из семинара или любые другие (можно брать только часть текста, если считается слишком долго). Перплексию рассчитывайте на 10-50 отложенных предложениях (они не должны использоваться при сборе статистик).


Подсказки:  
    - нужно будет добавить еще один тэг \<start>  
    - можете использовать тот же подход с матрицей вероятностей, но по строкам хронить биграмы, а по колонкам униграммы 
    - тексты должны быть очень похожи на нормальные (если у вас получается рандомная каша, вы что-то делаете не так)
    - у вас будут словари с индексами биграммов и униграммов, не перепутайте их при переводе индекса в слово - словарь биграммов будет больше словаря униграммов и все индексы из униграммного словаря будут формально подходить для словаря биграммов (не будет ошибки при id2bigram[unigram_id]), но маппинг при этом будет совершенно неправильным 

#### Читаем файлы

In [162]:
def read_file(filepath: str) -> str:
    with open(filepath, "r", encoding="utf-8") as file:
        data = file.read()
    return data

In [212]:
news_raw  = read_file(r"./data/lenta.txt")

#### Нормализация из лекции

In [164]:
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np

In [165]:
def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word \
                                                            in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text

#### Моделируем.

In [166]:
from nltk.tokenize import sent_tokenize
def ngrammer(tokens, n=2):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

In [167]:
sentences_news  = [['<start>', '<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(news_raw[:5000000] )]

In [168]:
from scipy.sparse import lil_matrix, csr_matrix, csc_matrix
from collections import Counter

In [169]:
class Model:

    def __init__(self, sentences: list[str]):
        
        unigrams = Counter()
        bigrams  = Counter()
        trigrams = Counter()

        for sentence in sentences:
            unigrams.update(sentence)
            bigrams .update(ngrammer(sentence))
            trigrams.update(ngrammer(sentence, 3))

        tmp_matrix = lil_matrix(
            (
                len(bigrams ),
                len(unigrams),
            )
        )

        self.id2unigram = list(unigrams)
        self.id2bigram  = list(bigrams )

        self.unigram2id = {unigram:i for i, unigram in enumerate(self.id2unigram)}
        self.bigram2id  = {bigram :i for i, bigram  in enumerate(self.id2bigram )}

        for trigram in trigrams:
            bigram, word = trigram.rsplit(maxsplit=1)
            tmp_matrix[self.bigram2id[bigram], self.unigram2id[word]] = trigrams[trigram] / bigrams[bigram]

        self.matrix = csc_matrix(tmp_matrix)

    @staticmethod
    def apply_temperature(probas, temperature):
        # логарифмирование и деление на температуру
        log_probas = np.log(np.maximum(probas, 1e-10))  
        adjusted_log_probas = log_probas / temperature
        # чтобы получить честные вероятности, нужно применить софтмакс
        exp_probas = np.exp(adjusted_log_probas)
        adjusted_probabilities = exp_probas / np.sum(exp_probas)
        return adjusted_probabilities

    def generate(self, n=100, start='<start> <start>', temperature=1):
        text = start.split()
        current_idx = self.bigram2id[start]

        for _ in range(n):

            chosen_idx = np.random.choice(
                self.matrix.shape[1],
                p = self.apply_temperature(
                    self.matrix[current_idx].toarray()[0],
                    temperature=temperature
                )
            )

            chosen = self.id2unigram[chosen_idx]
            text.append(chosen)

            if chosen == '<end>':
                current_idx = self.bigram2id['<start> <start>']
                text.extend(['<start>', '<start>'])
            else:
                current_idx = self.bigram2id[" ".join(text[-2:])]
                    
        
        return ' '.join(text)


In [170]:
lenta_model = Model(sentences_news)

In [171]:
print(
    lenta_model.generate(n=40)
        .replace("<start> <start>", "")
        .replace("<end>", "\n")
)

 делом занимаются три адвоката специалисты по финансовым операциям корреспондентские связи между операционной системой linux 
  в частности в калининградской области в качестве первоочередной экстренной меры олег миронов сообщил что талибы давно планировали установить с чечней мирных жителей из города наска


In [172]:
print(
    lenta_model.generate(n=40, temperature=0.01)
        .replace("<start> <start>", "")
        .replace("<end>", "\n")
)

 в настоящее время в россии 
  в настоящее время в составе делегации министр внутренних дел россии игорь иванов 
  в настоящее время в разработке кремлевских сценариев по понижению рейтинга избирательного блока отечество вся россия 
  в настоящее время в москве


In [173]:
print(
    lenta_model.generate(n=40, temperature=1.5)
        .replace("<start> <start>", "")
        .replace("<end>", "\n")
)

 56 человек погибли 7821 получили ранения 
  причем один из последних укрепрайонов боевиков на ближних подступах к грозному вообще подходить нельзя нужно забрать гражданских и оставить на потом 
  кроме мост-банка проверка проводится в отношении своего присутствия в приднестровье означает


In [180]:
# функции возвращают лог (чтобы проверить с первой функцией можно добавить np.exp(prob))
def compute_joint_proba(text, word_probas):
    prob = 0
    tokens = normalize(text)
    for word in tokens:
        if word in word_probas:
            prob += (np.log(word_probas[word]))
        else:
            prob += np.log(2e-4)
    
    return prob, len(tokens)

def perplexity(logp, N):
    return np.exp((-1/N) * logp)

In [176]:
fresh_corpus = news_raw[5000000:]

In [177]:
len(fresh_corpus)

6536552

In [178]:
norm_fresh = normalize(fresh_corpus)
vocab_fresh = Counter(norm_fresh)
probas_fresh = Counter({word:c/len(norm_fresh) for word, c in vocab_fresh.items()})

In [179]:
probas_fresh.most_common(10)

[('в', 0.04898059737390862),
 ('и', 0.021790300321547912),
 ('на', 0.019109566809343337),
 ('по', 0.012699624744010965),
 ('что', 0.010921420415140073),
 ('с', 0.010537290917964418),
 ('не', 0.00827104364131412),
 ('из', 0.005248213038919674),
 ('о', 0.004815045733593934),
 ('как', 0.0045873701957542595)]

In [184]:
texts = (
    lenta_model.generate(n=500)
        .replace("<start> <start>", "")
        .replace("<end>", "\n")
).split("\n")

In [185]:
texts

[' его звонок был выведен в прямой эфир нтв где в 1997 году когда церковьпыталась примирить противостоящие стороны а также выступать с более взвешенными заявлениями сегодня борис березовский зарегистрирован кандидатом в депутаты госдумы по 212 одномандатному округу ',
 '  антимонопольное ведомство минюста сша и 19 изобретений ',
 '  а с ',
 '  за минувшие сутки вертолеты федеральных войскнаносили удары по базам боевиков в грозном продолжаются ожесточенные бои федеральные войска пытаются окончательно замкнуть кольцо вокруг грозного ',
 '  как сообщает bbc в ночь на 31 что сделано с тем режим работы отметил министр ',
 '  в конечном счете в те же фамилии сказалибы что эти средства предполагается получить у владельца находки дальневосточной российской компании сообщает риа новости сообщили в гувд столицы с ходатайством о проверке всехпартий ядерных грузов поступивших в органы государственной власти и организаций прекрасно осведомлены о создавшейся ситуации необходимо вступить в контакты с

In [186]:
pps = []
for text in texts:
    pps.append(
        perplexity(*compute_joint_proba(
            text,
            probas_fresh
        ))
    )

In [189]:
np.mean(pps)

np.float64(4216.453570165646)

## Задание № 2* (2 балла). 

Измените функцию generate_with_beam_search так, чтобы она работала с моделью, которая учитывает два предыдущих слова. 
Сравните получаемый результат с первым заданием. 
Также попробуйте начинать генерацию не с нуля (подавая \<start> \<start>), а с какого-то промпта. Но помните, что учитываться будут только два последних слова, так что не делайте длинные промпты.

In [193]:
# сделаем класс чтобы хранить каждый из лучей
class Beam:
    def __init__(self, sequence: list, score: float):
        self.sequence: list = sequence
        self.score: float = score

In [202]:
def generate_with_beam_search(
    model: Model,
    n=100,
    max_beams=5,
    start='<start> <start>'
) :
    initial_node = Beam(sequence=start.split(), score=np.log1p(0))
    beams = [initial_node]
    for i in range(n):
        new_beams = []

        for beam in beams:

            if beam.sequence[-1] == '<end>':
                new_beams.append(beam)
                continue

            last_id = model.bigram2id[
                " ".join(beam.sequence[-2:])
            ]

            probas = model.matrix[last_id].toarray()[0]
            top_idxs = probas.argsort()[:-(max_beams+1):-1]

            for top_id in top_idxs:
                if not probas[top_id]:
                    break

                new_sequence = beam.sequence + [model.id2unigram[top_id]]

                new_score = (beam.score + np.log1p(probas[top_id])) / len(new_sequence)
                new_beam = Beam(sequence=new_sequence, score=new_score)
                new_beams.append(new_beam)

        beams = sorted(new_beams, key=lambda x: x.score, reverse=True)[:max_beams]
    
    sorted_sequences = sorted(beams, key=lambda x: x.score, reverse=True)
    sorted_sequences = [" ".join(beam.sequence) for beam in sorted_sequences]
    return sorted_sequences

In [203]:
generate_with_beam_search(lenta_model)

['<start> <start> как сообщает риа новости <end>',
 '<start> <start> об этом риа новости <end>',
 '<start> <start> об этом сообщает риа новости <end>',
 '<start> <start> об этом сообщает агентство риа новости <end>',
 '<start> <start> об этом сообщает итар-тасс со ссылкой на пресс-службу мчс россии сергей шойгу <end>']

In [204]:
generate_with_beam_search(lenta_model, start="<start> вечером")

['<start> вечером накануне ареста кузнецова <end>',
 '<start> вечером накануне ареста кузнецова милиция новокузнецка по приказу наполеона <end>',
 '<start> вечером мадлен олбрайт <end>',
 '<start> вечером накануне ареста кузнецова милиция новокузнецка по приказу губернатора кемеровской области амана тулеева в руководстве корпорацииакции microsoft стали повышаться что значительно дороже чем многоразовые <end>',
 '<start> вечером накануне ареста кузнецова милиция новокузнецка по приказу губернатора кемеровской области амана тулеева в руководстве корпорацииакции microsoft стали повышаться что значительно дороже чем и без того непростое экономическое положение компании <end>']

In [205]:
generate_with_beam_search(lenta_model, start="вечером на")

['вечером на мине направленного действия подорвалась легковая машина <end>',
 'вечером на мине оставленной немцами при отходе из севастополя <end>',
 'вечером на мине заложенной террористами подорвалсяавтобус с индийскими военнослужащими <end>',
 'вечером на экстренном заседании христианские демократы потребовали от баскских сепаратистов eta сообщает риа новости <end>',
 'вечером на экстренном заседании христианские демократы потребовали от баскских сепаратистов eta сообщает риа новости со ссылкой на пресс-службу мчс россии сергей шойгу <end>']

In [206]:
generate_with_beam_search(lenta_model, start="я люблю маму")

KeyError: 'люблю маму'

In [210]:
generate_with_beam_search(lenta_model, start="в ходе переговоров")

['в ходе переговоров речь пойдет о развитии газификации <end>',
 'в ходе переговоров посвященных развитию сотрудничества правоохранительных органов <end>',
 'в ходе переговоров речь пойдет о юридическом закреплении в хартии югославского прецедента когда международное сообщество и наша страна лежит в руинах <end>',
 'в ходе переговоров речь пойдет о юридическом закреплении в хартии югославского прецедента когда международное сообщество и наша страна поймут что эти люди ждали справедливости 50 лет <end>',
 'в ходе переговоров речь пойдет о юридическом закреплении в хартии югославского прецедента когда международное сообщество и наша страна лежит в руинах выживших родных и близких <end>']